In [1]:
!pip install jiwer -qq

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('..')

In [4]:
from predict import predict
import onnxruntime
import onnx
from pathlib import Path
import jiwer
import pandas as pd
from utils import get_files
from pdb import set_trace
from words2num import normalize
pd.set_option('max_colwidth', 400)

In [5]:
model = onnx.load("/content/output.onnx")
model = onnxruntime.InferenceSession(model.SerializeToString())

In [6]:
data_path = "/content/test-bahasa/"

In [7]:
txts = get_files(data_path,[".txt"],recurse=True)
txts

(#780) [PosixPath('/content/test-bahasa/wattpad-audio-wattpad-648.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-news-audio-403.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-streaming-berakhir dengan kekalahan perancis.txt'),PosixPath('/content/test-bahasa/wattpad-audio-wattpad-102.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-news-audio-254.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ183-000005.txt'),PosixPath('/content/test-bahasa/iium-audio-iium-87.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-amangkurat.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ241-000005.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-akronim.txt')...]

In [8]:
# find data with number in it
nombor = ["satu","dua",'tiga','puluh','ratus','ribu']
filtered_wavs = []
for i in txts:
    text = i.read_text().split()
    got = any([i in nombor for i in text])
    if got:
        filtered_wavs.append(i.with_suffix(".wav"))

# Predict single or multiple file

In [9]:
preds,files,entropy = predict(model,filtered_wavs)

100%|██████████| 37/37 [00:24<00:00,  1.50it/s]


In [10]:
data = []
for i,j in zip(preds,files):
    label = j.with_suffix('.txt').read_text()
    data.append([label,i,normalize(i)])

Invalid number state from D to D dua puluh enam empat


In [11]:
df = pd.DataFrame(data)
df.columns = ["label","pred","norm_pred"]
df.head(10)

,label,pred,norm_pred
0,wong menyifatkan hantaran facebook itu sebagai satu cara,wang menyifartkan hantaran facebook itu sebagai satu cara,wang menyifartkan hantaran facebook itu sebagai 1 cara
1,di kamar dengan dua tempat tidur,di kamar dengan dua tempat tidur,di kamar dengan 2 tempat tidur
2,studi ini juga mendapati sebanyak empat puluh tiga percent anak yang datang,studi ini juga mendapati sebanyak empat puluh tiga persen anak yang datang,studi ini juga mendapati sebanyak 43 persen anak yang datang
3,kenapa pembenci muslihat membuat kebohongan tanpa satu bukti,kenapa pembenci muslihat membuat kebohongan tanpa satu bukti,kenapa pembenci muslihat membuat kebohongan tanpa 1 bukti
4,diikuti siswa siswi kelas sebelas ipa tiga dibelakangnya termasuk libra dan geng,diikuti siswa siswi kelas sebelas ip tiga di belakangnya termasuk libra,diikuti siswa siswi kelas 11 ip 3 di belakangnya termasuk libra
5,gila cape banget kecepatan lari kamu kalau lagi panik nambah dua kali,kilah capit bangyit kecapatan lari kamu kalau lagi panik enambah dua kali,kilah capit bangyit kecapatan lari kamu kalau lagi panik enambah 2 kali
6,empat puluh lima ahli belia yang dilantik itu menjadi penyambung legasi,empat puluh lima ahli belia yang dilantik itu menjadi penyambung legasi,45 ahli belia yang dilantik itu menjadi penyambung legasi
7,mereka berjalan menyusuri koridor kelas dua belas atau biasa disebut jalan pintas,mereka berjalan menyusuri koridor kelas dua belas atau biasa disebut jalan pintas,mereka berjalan menyusuri koridor kelas 12 atau biasa disebut jalan pintas
8,aku harap tak adalah orang racist komen kat sini pasal indo dan kenapa banyak dua negara mesti nak pilih indo pulak,aku harap tak adalah orang racis komen kat sini pasalan indo dan kenapa banyak dua negri,aku harap tak adalah orang racis komen kat sini pasalan indo dan kenapa banyak 2 negri
9,adakah ia ok kerana seorang lelaki berusia tiga puluh tujuh tahun,adakah ia okey kerana seorang lelaki berusia tiga puluh tujuh tahun,adakah ia okey kerana seorang lelaki berusia 37 tahun


In [12]:
# word to num normalization still wip, let use know if found a bug so we can fix it asap
# TODO: fix ip address digit spelling